<a href="https://colab.research.google.com/github/sean25benhur/catsvsdogs/blob/master/Cats_vs_Dogs_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U tensorflow_datasets

     |████████████████████████████████| 3.3MB 88kB/s 
  Found existing installation: tensorflow-datasets 2.1.0
    Uninstalling tensorflow-datasets-2.1.0:
      Successfully uninstalled tensorflow-datasets-2.1.0


In [0]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import layers

In [6]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteIKDRDV/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image,label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [14]:
!pip install tensorflow_hub

In [0]:
import tensorflow_hub as hub
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape = (IMAGE_RES,IMAGE_RES,3))

In [0]:
feature_extractor.trainable = False

In [18]:
model = tf.keras.Sequential([
                             feature_extractor,
                             layers.Dense(2)                           
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [20]:
model.compile(
    optimizer = 'adam',
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    validation_data = validation_batches)

Epoch 1/3
582/582 [==============================] - 33s 57ms/step - loss: 0.0536 - accuracy: 0.9817 - val_loss: 0.0347 - val_accuracy: 0.9888
Epoch 2/3
582/582 [==============================] - 20s 35ms/step - loss: 0.0296 - accuracy: 0.9895 - val_loss: 0.0307 - val_accuracy: 0.9893
Epoch 3/3
582/582 [==============================] - 20s 35ms/step - loss: 0.0243 - accuracy: 0.9925 - val_loss: 0.0329 - val_accuracy: 0.9893


In [21]:
class_names = np.array(info.features['label'].names)
class_names

array(['cat', 'dog'], dtype='<U3')

In [22]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

array(['dog', 'dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'cat', 'dog',
       'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'cat', 'dog',
       'dog', 'dog', 'cat', 'cat', 'dog', 'cat', 'cat', 'cat', 'dog',
       'dog', 'dog', 'dog', 'cat', 'cat'], dtype='<U3')

In [23]:
print('Labels:', label_batch)
print('Predicted_labels:', predicted_ids)

Labels: [1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0]
Predicted_labels: [1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0]


In [29]:
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1587569113.h5


In [30]:
!ls

1587569113.h5  drive  sample_data


In [31]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [33]:
(abs(result_batch - reloaded_result_batch)).max()

0.0

In [34]:
EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 21s 35ms/step - loss: 0.0212 - accuracy: 0.9924 - val_loss: 0.0321 - val_accuracy: 0.9899
Epoch 2/3
582/582 [==============================] - 20s 35ms/step - loss: 0.0189 - accuracy: 0.9930 - val_loss: 0.0341 - val_accuracy: 0.9905
Epoch 3/3
582/582 [==============================] - 20s 35ms/step - loss: 0.0175 - accuracy: 0.9937 - val_loss: 0.0337 - val_accuracy: 0.9905


In [35]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

./1587569363
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./1587569363/assets


INFO:tensorflow:Assets written to: ./1587569363/assets


In [36]:
!ls {export_path_sm}

assets	saved_model.pb	variables


In [0]:
reloaded_sm = tf.saved_model.load(export_path_sm)

In [0]:
reload_sm_result_batch = reloaded_sm(image_batch, training=False).numpy()

In [40]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)
tf.saved_model.save(model, export_path_sm)

./1587569696
INFO:tensorflow:Assets written to: ./1587569696/assets


INFO:tensorflow:Assets written to: ./1587569696/assets


In [41]:
reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
result_batch = model.predict(image_batch)
reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)

In [43]:
(abs(result_batch - reload_sm_keras_result_batch)).max()

0.0

In [44]:
!zip -r model.zip {export_path_sm}

  adding: 1587569696/ (stored 0%)
  adding: 1587569696/saved_model.pb (deflated 92%)
  adding: 1587569696/assets/ (stored 0%)
  adding: 1587569696/variables/ (stored 0%)
  adding: 1587569696/variables/variables.data-00000-of-00002 (deflated 88%)
  adding: 1587569696/variables/variables.data-00001-of-00002 (deflated 7%)
  adding: 1587569696/variables/variables.index (deflated 79%)


In [45]:
!ls

1587569113.h5  1587569363  1587569696  drive  model.zip  sample_data


In [46]:
try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass

MessageError: ignored